# ĐỒ ÁN CUỐI KÌ

Bùi Hữu Nghĩa - 1712613

## TỔNG QUAN

Qui trình KHDL:
- Đưa ra câu hỏi
- Thu thập dữ liệu
- Khám phá dữ liệu
- Tiền xử lí
- Phân tích, trả lời câu hỏi

## 1 - THU THẬP DỮ LIỆU

Ở đây em có dữ liệu là thông tin về các ứng dụng/trò chơi ở trên Steam  
Dữ liệu được lấy từ trang [kaggle.com](https://www.kaggle.com)  
Thông tin về dữ liệu tại [đây](https://www.kaggle.com/nikdavis/steam-store-games?select=steam.csv)  
Người sở hữu: Nik Davis  
License: **Attribution 4.0 International (CC BY 4.0)**  
Dữ liệu được tác giả thu thập từ SteamStore và SteamSpy APIs

## 2 - KHÁM PHÁ DỮ LIỆU

Trước khi đưa ra câu hỏi để trả lời thì cần phải biết dữ liệu có gì. Vì vậy, đầu tiên ta cần đọc dữ liệu

In [31]:
import numpy as np
import pandas as pd
import matplotlib as plt
import re

Dữ liệu lưu trữ thông tin của hơn 27 nghìn app/game, ở đây phần lớn là game nên em sẽ gọi tắt là game   
Để biểu diễn trực quan dữ liệu, em dùng Dataframe của pandas

In [2]:
games_df = pd.read_csv('steam.csv')

In [3]:
games_df.shape

(27075, 18)

Vậy dữ liệu có kích thước là 27075 dòng x 18 cột

Xem thử thông tin các game trông như thế nào

In [4]:
games_df.head()

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99


Mỗi dòng là thông tin của một game

Thông tin về các cột như sau:  
- **appid**  : là id của ứng dụng
- **name**: tên game
- **release_date**: ngày phát hành
- **english**: ngôn ngữ hỗ trợ, 1 nếu là tiếng anh
- **developer**: nhà phát triển
- **publisher**: nhà phát hành
- **platforms**: nền tảng(game hỗ trợ nền tảng nào, như Window, MAC, Linux)
- **required_age**: độ tuổi yêu cầu
- **categories**: loại trò chơi(yêu cầu) như chơi một hay nhiều người, đa nền tảng,... 
- **genres**: thể loại trò chơi(nội dung) như hành động, chiến thuật,...
- **steamspy_tags**: giống như **genres**
- **achievements**: thành tựu trong game(nếu có)
- **positive_ratings/negative_ratings**: rating của game
- **average_playtime**: thời gian chơi trung bình
- **median_playtime**: thời gian chơi trung vị
- **owners**: số người sở hữu
- **price**: giá game(tính theo bảng Anh, British Pound Sterling - GBP)

Xem thử kiểu dữ liệu của các cột dữ liệu

In [5]:
games_df.dtypes

appid                 int64
name                 object
release_date         object
english               int64
developer            object
publisher            object
platforms            object
required_age          int64
categories           object
genres               object
steamspy_tags        object
achievements          int64
positive_ratings      int64
negative_ratings      int64
average_playtime      int64
median_playtime       int64
owners               object
price               float64
dtype: object

Như trên thì hầu hết các cột đều có kiểu dữ liệu phù hợp trừ cột **release_date(ngày phát hành)**. Việc tiếp theo cần làm là chuyển kiểu dữ liệu của **release_date** từ **object** sang **datetime**

In [20]:
games_df['release_date'] = pd.to_datetime(games_df['release_date'], format="%Y-%m-%d")

Kiểm tra lại cột release_date đã đổi kiểu dữ liệu chưa

In [21]:
games_df.dtypes

appid                        int64
name                        object
release_date        datetime64[ns]
english                      int64
developer                   object
publisher                   object
platforms                   object
required_age                 int64
categories                  object
genres                      object
steamspy_tags               object
achievements                 int64
positive_ratings             int64
negative_ratings             int64
average_playtime             int64
median_playtime              int64
owners                      object
price                      float64
dtype: object

Vậy là ta đã đổi kiểu dữ liệu thành công, có thể thao tác tới cột release_date.  
Bây giờ ta cần tính miền giá trị của các cột dữ liệu có dạng **numeric** hoặc **datetime** như: **appid, release_date, required_age, achievement, positive_ratings, negative_ratings, average_playtime, median_playtime, price**  
Có thể bỏ qua cột **english** vì chỉ có 2 giá trị là 0 và 1

In [22]:
def cal_value_field(cols):
    for col in cols:
        print('Miền giá trị của', col, 'là: [', min(games_df[col]), ',', max(games_df[col]), ']')

In [28]:
cal_value_field([
    'appid',
    'release_date', 
    'required_age',
    'achievements',
    'positive_ratings',
    'negative_ratings',
    'average_playtime',
    'median_playtime',
    'price'
])

Miền giá trị của appid là: [ 10 , 1069460 ]
Miền giá trị của release_date là: [ 1997-06-30 00:00:00 , 2019-05-01 00:00:00 ]
Miền giá trị của required_age là: [ 0 , 18 ]
Miền giá trị của achievements là: [ 0 , 9821 ]
Miền giá trị của positive_ratings là: [ 0 , 2644404 ]
Miền giá trị của negative_ratings là: [ 0 , 487076 ]
Miền giá trị của average_playtime là: [ 0 , 190625 ]
Miền giá trị của median_playtime là: [ 0 , 190625 ]
Miền giá trị của price là: [ 0.0 , 421.99 ]


## 3 - CÁC CÂU HỎI CẦN ĐƯỢC TRẢ LỜI TỪ DỮ LIỆU

- Game nào có doanh thu cao nhất
- So sánh các nền tảng phát hành game
- Game thể loại nào được chơi nhiều nhất
- Nhà phát hành game nào có nhiều thành tựu trong game nhất

## 4 - TRẢ LỜI CÂU HỎI

### Game nào có doanh thu cao nhất

Ở đây ta sẽ tính doanh thu của các game rồi chọn ra game có doanh thu cao nhất. Tuy nhiên ở tập dữ liệu này, cột **owners** tức người sỡ hữu chỉ là ước tính, có dạng lower_bound:upper_bound. Giả sử số người mua là trung bình của lower và upper thì doanh thu của game sẽ là revenue = (lower + upper) / 2 * price

Trong cột **owners** có dạng lower-upper nên em dùng regex để tách lấy 2 giá trị  
Tính cột doanh thu

In [68]:
txt = "0-200"
x = re.findall(r"\d+", txt)
a, b = x
print(type(a), b)

<class 'str'> 200


In [70]:
np.mean([2, 3])

2.5

In [81]:
# tạo cột doanh thu
games_df['revenue'] = games_df['owners']

# hàm tách giá trị từ owner rồi tính trung bình
def cal_average(owner): 
    arr = re.findall(r"\d+", owner)
    return np.mean([int(arr[0]), int(arr[1])])

# apply hàm ở trên vào cột revenue ta được cột revenue có giá trị là trung bình của lower và upper trong owners
games_df['revenue'] = games_df['revenue'].apply(cal_average)

# nhân cột revenue với cột price ta được doanh thu tính theo bảng Anh
games_df['revenue'] = games_df['revenue'] * games_df['price']

# tìm dòng có revenue lớn nhất
games_df.loc[games_df['revenue'] == max(games_df['revenue'])]

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price,revenue
12836,578080,PLAYERUNKNOWN'S BATTLEGROUNDS,2017-12-21,1,PUBG Corporation,PUBG Corporation,windows,0,Multi-player;Online Multi-Player;Stats,Action;Adventure;Massively Multiplayer,Survival;Shooter;Multiplayer,37,496184,487076,22938,12434,50000000-100000000,26.99,2.024250e+09


In [84]:
games_df.iloc[12836, 18]

2024250000.0

Vậy game có doanh thu cao nhất là **PLAYERUNKNOWN'S BATTLEGROUNDS** với doanh thu là $2,024,250,000$ bảng Anh

### So sánh các nền tảng phát hành game

### Game thể loại nào được chơi nhiều nhất

### Nhà phát hành game nào có nhiều thành tựu trong game nhất

## 5 - NHÌN LẠI QUÁ TRÌNH THỰC HIỆN

## 6 - TÀI LIỆU THAM KHẢO

- [kaggle.com](https://www.kaggle.com/) để lấy dữ liệu
- Thư viện [Pandas](https://pandas.pydata.org/pandas-docs/stable/index.html)
- Các demo của thầy trong môn LTCKHDL